In [277]:
import numpy as np
import cv2
import sys

pathImage = input("Путь до изображения: ")

# Загружаем и клонируем изображение для работы с ним
image = cv2.imread(pathImage)
clone = image.copy()
cv2.imwrite("result/clone.bmp", clone)

True

HSV (англ. Hue, Saturation, Value — тон, насыщенность, значение) — цветовая модель, в которой координатами цвета являются:
    
* Hue — цветовой тон, (например, красный, зелёный или сине-голубой). Варьируется в пределах 0—360°, однако иногда приводится к диапазону 0—100 или 0—1.
* Saturation — насыщенность. Варьируется в пределах 0—100 или 0—1. Чем больше этот параметр, тем «чище» цвет, поэтому этот параметр иногда называют чистотой цвета. А чем ближе этот параметр к нулю, тем ближе цвет к нейтральному серому.
* Value (значение цвета) или Brightness — яркость. Также задаётся в пределах 0—100 и 0—1.
    
Эта модель очень удобна для поиска на изображении объектов по цвету (и яркости). Например, объекты с выделяющимся цветом.

In [279]:
# Меняем цветовую модель с BGR на HSV
hsv = cv2.cvtColor(clone, cv2.COLOR_BGR2HSV)
cv2.imwrite("result/hsv.bmp", hsv)

True

Формируем бинарное изображение, которое получается из тех пикселей канала, значение которых больше hsv_min и меньше hsv_max.

In [280]:
# Параметры цветового фильтра
hsv_min = np.array((7, 120, 0), np.uint8)
hsv_max = np.array((15, 255, 200), np.uint8)

cont = cv2.inRange(hsv, hsv_min, hsv_max)
cv2.imwrite("result/cont.bmp", cont)

True

Подход учитывает только центры кругов. 

In [281]:
seed_pt = (25, 25)
fill_color = 0
mask = np.zeros_like(cont)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

prev_mask = mask.copy()
mask = cv2.threshold(cont, 0, 255, cv2.THRESH_BINARY)[1]
mask = cv2.floodFill(mask, None, seed_pt, fill_color)[1]
mask = cv2.bitwise_or(mask, prev_mask)
mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

n_centers = cv2.connectedComponentsWithStats(mask)[0] - 1
n_centers1 = cv2.connectedComponentsWithStats(mask)[3]

print("%d необрушенных зёрен на картинке."%n_centers)

325 необрушенных зёрен на картинке.


Вывод на картинку местоположение необрушенных зёрен.

In [282]:
k = 1
for i in n_centers1:
    cv2.putText(clone, str(k), (int(i[0]), int(i[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 3)
    k += 1
cv2.imwrite("result/clone.bmp", clone)

True